In [2]:
# import everything we may need
import mysql.connector
from mysql.connector import errorcode
import config
import requests
import time
import json
import pandas as pd
%load_ext autoreload
%autoreload 2
import json

In [66]:
#get data from box office mojo as json file
f = open('mojo_data.json')
data = json.load(f)

In [8]:
# connect to AWS
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)

# Establish cursor
cursor = cnx.cursor()

In [4]:
## function to close the connection
def close_connections():
    cursor.close()
    conn.close()

In [5]:
#name the db
db_name = 'Mod_1_Project'

In [6]:
# create database in AWS
def create_database(cursor, database_name):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
    except mysql.connector.Error as err:
        ## Catch the error if an error occurs.
        print("Failed creating database: {}".format(err))
        exit(1)
        
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [67]:
#get list of movies from box office mojo data
titles = [x['title'] for x in data]
titles

['The Boss Baby',
 "A Dog's Purpose",
 'Jigsaw',
 'Guardians of the Galaxy Vol. 2',
 'Split',
 'Jumanji:Welcome to the Jungle',
 'All Eyez on Me',
 'Downsizing',
 'Annabelle:Creation',
 'Despicable Me 3',
 'Spider-Man:Homecoming',
 "Molly's Game",
 'The LEGO Batman Movie',
 'American Assassin',
 'Geostorm',
 'Star Wars:The Last Jedi',
 'Transformers:The Last Knight',
 'Baby Driver',
 'Ghost in the Shell (2017)',
 'Atomic Blonde',
 'Beauty and the Beast (2017)',
 'Justice League',
 'Logan Lucky',
 "Tyler Perry's Boo 2! A Madea Halloween",
 'Rings',
 'Victoria and Abdul',
 'Smurfs:The Lost Village',
 'I, Tonya',
 'Baywatch',
 'The Shape of Water',
 'Rough Night',
 'Thor:Ragnarok',
 'Valerian and the City of a Thousand Planets',
 'Alien:Covenant',
 'American Made',
 'Captain Underpants:The First Epic Movie',
 'The Dark Tower',
 'Get Out',
 'Lady Bird',
 'Coco',
 'The Post',
 'John Wick:Chapter Two',
 'The Foreigner',
 'The Fate of the Furious',
 'Leap!',
 'Kidnap (2017)',
 'Pirates of the

In [69]:
key = config.apiKey
url = 'http://www.omdbapi.com/?apikey='

# Make call to API
# Iterate over list of titles (from box office mojo) to retrieve all

def OMDB_call(titles):
    all_titles = []
    for title in titles: 
        parameters = {'t': {title}}
        response = requests.get(url+key, params=parameters)
        data = response.json()
        all_titles.append(data)
        time.sleep(.5)
    return all_titles

movie_dict = OMDB_call(titles[:100])

In [71]:
movie_dict[0]

{'Title': 'The Boss Baby',
 'Year': '2017',
 'Rated': 'PG',
 'Released': '31 Mar 2017',
 'Runtime': '97 min',
 'Genre': 'Animation, Adventure, Comedy, Family, Fantasy',
 'Director': 'Tom McGrath',
 'Writer': 'Michael McCullers, Marla Frazee (based on the book by)',
 'Actors': 'Alec Baldwin, Steve Buscemi, Jimmy Kimmel, Lisa Kudrow',
 'Plot': 'A suit-wearing, briefcase-carrying baby pairs up with his 7-year old brother to stop the dastardly plot of the CEO of Puppy Co.',
 'Language': 'English, Spanish',
 'Country': 'USA',
 'Awards': 'Nominated for 1 Oscar. Another 1 win & 19 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg5MzUxNzgxNV5BMl5BanBnXkFtZTgwMTM2NzQ3MjI@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.3/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '52%'},
  {'Source': 'Metacritic', 'Value': '50/100'}],
 'Metascore': '50',
 'imdbRating': '6.3',
 'imdbVotes': '94,906',
 'imdbID': 'tt3874544',
 'Type': 'movie',
 'DVD': '25 Jul

In [ ]:
#function to create tables in our database
def create_table(dict_of_tables, db_name):
    for table_name in dict_of_tables:
        table_query = dict_of_tables[table_name]
        cursor.execute("USE {}".format(db_name))
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(table_query)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")


In [46]:
#Create tables to input data
TABLES = {}
TABLES['main_movie_table'] = (
    "CREATE TABLE main_movie_table ("
    "  movie_id int NOT NULL AUTO_INCREMENT,"
    "  title text NOT NULL,"
    "  budget int(10),"
    "  release_date date,"
    "  award_wins int(3),"
    "  award_nomination int(3),"
    "  RT_rating int(2),"
    "  IMDB_rating real,"
    "  PRIMARY KEY (movie_id)"
    ") ENGINE=InnoDB")

TABLES['genres'] = (
    "CREATE TABLE genres ("
    "  genre_id int NOT NULL,"
    "  genre text,"
    "  PRIMARY KEY (genre_id)"
    ") ENGINE=InnoDB")

TABLES['key_words'] = (
    "CREATE TABLE key_words ("
    "  keyword_id int NOT NULL,"
    "  key_words text,"
    "  PRIMARY KEY (keyword_id)"
    ") ENGINE=InnoDB")

TABLES['movies_genres'] = (
    "CREATE TABLE movies_genres ("
      " movie_id int NOT NULL,"
      " genre_id int NOT NULL"
      ") ENGINE=InnoDB")

TABLES['movies_keywords'] = (
    "CREATE TABLE movies_keywords ("
      " movie_id int NOT NULL,"
      " keyword_id text NOT NULL"
      ") ENGINE=InnoDB")

TABLES['actors'] = (
    "CREATE TABLE actors ("
    " actor_id int NOT NULL AUTO_INCREMENT," 
    " actor_name text NOT NULL,"
    " PRIMARY KEY (actor_id)"
    ") ENGINE=InnoDB")

TABLES['movie_actors'] = (
    "CREATE TABLE movie_actors("
    " movie_id int NOT NULL,"
    " actor_id int NOT NULL"
    ") ENGINE=InnoDB")

create_table(TABLES, 'Mod_1_Project')

Creating table main_movie_table: OK
Creating table genres: already exists.
Creating table key_words: already exists.
Creating table movies_genres: already exists.
Creating table movies_keywords: already exists.
Creating table actors: already exists.
Creating table movie_actors: already exists.


In [ ]:
#insert data into tables
def add_movies_toDB():
    for movie in movie_dict:
        ('''insert into main_movie_table (title, release_date, award_wins, award_nomination, 
        RT_rating, IMDB_rating) values(%s,%s,%s,%s,%s,%s)''',
        movie['Title'], movie['Released'], movie['Awards'], 
        movie['Rating'][1]['Value'].replace('%', ''), movie['imdbRating']) 
        conn.commit
         
def add_genres_toDB():
    for movie in movie_dict:
        ('''insert into genres (genre) values(%s)''', movie['Genre'])
        conn.commit
            
## Drop all tables to refresh
def drop_everything():
    cursor.execute("""DROP TABLE Reviews;""")
    cnx.commit()
    cursor.execute("""DROP TABLE Businesses;""")
    cnx.commit()

In [21]:
import sys
def get_movie_id(title):
    title = title.replace(r"'",r"\'")
    cursor.execute(f"""SELECT movie_id FROM main_movie_table WHERE title = '{title}';""")
    return cursor.fetchone()

def add_actors_toDB():
    idx = 0
    sys.stdout.write("INSERTING ACTORS")
    sys.stdout.flush()
    for movie in movie_dict:
        try:
            for actor in movie['Actors'].split(', '):
                actor = actor.replace(r"'",r"\'")
                actor_insert_q = f"""INSERT INTO actors(actor_name) VALUES('{actor}');"""
                #get id of movie
                movie_id = get_movie_id(movie['Title'])
                #get actor_id if already present in database
                cursor.execute(f"""SELECT actor_id FROM actors WHERE actor_name = '{actor}';""")
                actor_id = cursor.fetchone()
                #if actor not present, insert and get id back
                if actor_id is None:
                    cursor.execute(actor_insert_q)
                    cursor.execute("""SELECT LAST_INSERT_ID();""")
                    actor_id = cursor.fetchone()
                    cnx.commit()
                #add actor + movie to movie_actors
                cursor.execute(f"""INSERT INTO movie_actors(movie_id, actor_id) VALUES ('{movie_id}', '{actor_id}');""")
                cnx.commit()
                idx += 1
                if idx % 112 == 0:
                    sys.stdout.write(".")
                    sys.stdout.flush()
        except KeyError:
            print(movie)
    sys.stdout.write("OK")
    sys.stdout.flush()
add_actors_toDB()  

INSERTING ACTORS{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
.{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
{'Response': 'False', 'Error': 'Movie not found!'}
.{'Response': 'False', 'Error': 'Movie not found!'}
.OK

In [11]:
#insert keywords into database
movie_keywords = pd.read_csv('improved_import_movies.csv')
keywords = movie_keywords['keyword'].unique()
cursor.execute("""USE Mod_1_Project;""")
for keyword in keywords:
    insert_keyword_q = f"""INSERT INTO key_words(key_words) VALUES ('{keyword}');"""
    cursor.execute(insert_keyword_q)
    cnx.commit()

In [16]:
from urllib.parse import urlencode
movie_keywords = pd.read_csv('improved_import_movies.csv')
movie_keywords = movie_keywords[movie_keywords['release_date'] < '2017-01-01']
titles = movie_keywords['title']
expanded_movie_data = []
url_title = urlencode({'t':titles[0]})
url_stem = 'http://www.omdbapi.com/?apikey=8f365774&'
omdb_data = json.loads(requests.get(url_stem+url_title).text)
award_wins = omdb_data['Awards']

expanded_movie_data.append({
    'title':cnx.escape_string(title),
    'award_wins': ,
    'award_nomination': ,
    'RT_rating': [rating['Value'][:-1] for rating in omdb_data['Ratings'] if rating['Source'] == 'Rotten Tomatoes']
    'IMDB_rating': [rating['Value'][:3] for rating in omdb_data['Ratings'] if rating['Source'] == 'Internet Movie Database']
# "  title text NOT NULL,"
#     "  budget int(10),"
#     "  release_date date,"
#     "  award_wins int(3),"
#     "  award_nomination int(3),"
#     "  RT_rating int(2),"
#     "  IMDB_rating real,"

{'Title': 'Forrest Gump',
 'Year': '1994',
 'Rated': 'PG-13',
 'Released': '06 Jul 1994',
 'Runtime': '142 min',
 'Genre': 'Drama, Romance',
 'Director': 'Robert Zemeckis',
 'Writer': 'Winston Groom (novel), Eric Roth (screenplay)',
 'Actors': 'Tom Hanks, Rebecca Williams, Sally Field, Michael Conner Humphreys',
 'Plot': 'The presidencies of Kennedy and Johnson, the events of Vietnam, Watergate, and other history unfold through the perspective of an Alabama man with an IQ of 75.',
 'Language': 'English',
 'Country': 'USA',
 'Awards': 'Won 6 Oscars. Another 40 wins & 67 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BNWIwODRlZTUtY2U3ZS00Yzg1LWJhNzYtMmZiYmEyNmU1NjMzXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.8/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '72%'},
  {'Source': 'Metacritic', 'Value': '82/100'}],
 'Metascore': '82',
 'imdbRating': '8.8',
 'imdbVotes': '1,622,006',
 'imdbID': 'tt0109830',
 'T

In [18]:
apiKey = config.apiKey
for title in titles:
    url_title = urlencode({'t':title})
    url_stem = 'http://www.omdbapi.com/?apikey='+apiKey+'&'
    omdb_data = json.loads(requests.get(url_stem+url_title).text)
    print(omdb_data['Awards'])

Won 3 Oscars. Another 51 wins & 197 nominations.
Won 1 Oscar. Another 122 wins & 181 nominations.


In [74]:
from bs4 import BeautifulSoup
from imdb import IMDb
import inspect

for movie_title in titles:
    movie = ia.search_movie(movie_title)[0]
    url_stem = 'https://www.imdb.com/title/tt'
    soup = BeautifulSoup(requests.get(url_stem+movie.getID()+'/awards').text)
    awards_text = soup.select('#main .desc')[0].string
    awards_words = awards_text.split()
    print({awards_words[3]:int(awards_words[2]), awards_words[6]:int(awards_words[5])})

{'wins': 4, 'nominations': 23}
{'wins': 0, 'nominations': 3}
{'wins': 8, 'nominations': 10}
{'wins': 14, 'nominations': 51}
{'wins': 9, 'nominations': 20}
{'wins': 5, 'nominations': 14}
{'wins': 0, 'nomination': 1}
{'win': 1, 'nominations': 16}
{'win': 1, 'nominations': 6}
{'wins': 0, 'nominations': 32}
{'wins': 7, 'nominations': 8}
{'wins': 7, 'nominations': 49}
{'wins': 12, 'nominations': 69}
{'wins': 0, 'nominations': 2}
{'wins': 0, 'nomination': 1}
{'wins': 22, 'nominations': 90}
{'wins': 2, 'nominations': 19}
{'wins': 41, 'nominations': 61}
{'wins': 2, 'nominations': 6}
{'wins': 10, 'nominations': 17}
{'wins': 15, 'nominations': 72}
{'wins': 2, 'nominations': 13}
{'wins': 2, 'nominations': 9}
{'win': 1, 'nominations': 3}
{'wins': 2, 'nominations': 0}
{'wins': 3, 'nominations': 13}
{'wins': 0, 'nominations': 5}
{'wins': 43, 'nominations': 119}
{'wins': 5, 'nominations': 14}
{'wins': 129, 'nominations': 336}
{'win': 1, 'nomination': 1}
{'wins': 6, 'nominations': 47}
{'wins': 0, 'nom

IndexError: list index out of range

In [73]:
titles

['The Boss Baby',
 "A Dog's Purpose",
 'Jigsaw',
 'Guardians of the Galaxy Vol. 2',
 'Split',
 'Jumanji:Welcome to the Jungle',
 'All Eyez on Me',
 'Downsizing',
 'Annabelle:Creation',
 'Despicable Me 3',
 'Spider-Man:Homecoming',
 "Molly's Game",
 'The LEGO Batman Movie',
 'American Assassin',
 'Geostorm',
 'Star Wars:The Last Jedi',
 'Transformers:The Last Knight',
 'Baby Driver',
 'Ghost in the Shell (2017)',
 'Atomic Blonde',
 'Beauty and the Beast (2017)',
 'Justice League',
 'Logan Lucky',
 "Tyler Perry's Boo 2! A Madea Halloween",
 'Rings',
 'Victoria and Abdul',
 'Smurfs:The Lost Village',
 'I, Tonya',
 'Baywatch',
 'The Shape of Water',
 'Rough Night',
 'Thor:Ragnarok',
 'Valerian and the City of a Thousand Planets',
 'Alien:Covenant',
 'American Made',
 'Captain Underpants:The First Epic Movie',
 'The Dark Tower',
 'Get Out',
 'Lady Bird',
 'Coco',
 'The Post',
 'John Wick:Chapter Two',
 'The Foreigner',
 'The Fate of the Furious',
 'Leap!',
 'Kidnap (2017)',
 'Pirates of the